# Transform WOA observational climatologies to potential density ($\sigma_0$) space

## The raw data can be found here:
    */tigress/GEOCLIM/LRGROUP/shared_data/salt_WOA/woa13_decav_s00_04v2.nc
    */tigress/GEOCLIM/LRGROUP/shared_data/temp_WOA/woa13_decav_t00_04v2.nc
    */tigress/GEOCLIM/LRGROUP/shared_data/oxygen_WOA/woa13_all_o00_01.nc

In [11]:
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

import warnings

import numpy as np
import xarray as xr

from fastjmd95 import rho
from dask.diagnostics import ProgressBar


from xarrayutils.file_handling import (
    write,
    maybe_create_folder,
)


from cmip6_omz.upstream_stash import (
    transform_wrapper,
)
from cmip6_omz.omz_tools import (
    omz_thickness,
    sigma_bins,
    align_missing,
)
from cmip6_omz.datasets import woa13

In [12]:
foldername = "fine_density_WOA"
ofolder = maybe_create_folder(f"../../data/external/{foldername}")

# global parameters
o2_bins = np.array([10, 40, 60, 80, 100, 120])
fine_sigma_bins = sigma_bins()

/home/samjd/anaconda3/envs/cmip6_omz2/lib/python3.9/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/external/fine_density_WOA does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


In [13]:
#read in WOA 2013 dataset
#NOTE: woa13 function only works for princeton tigress server!
woa = woa13().drop('time')

/projects/GEOCLIM/LRGROUP/samjd/global_omz/cmip6_omz/notebooks/sam_tests/../../cmip6_omz/datasets.py:33: UserWarning: No bounds found for lon and lat. Reconstructing with a very simplified method. Check results carefully.
  warnings.warn(


In [14]:
#compute potential density (sigma_0)
woa['sigma_0'] = (rho(woa.so, woa.thetao, 0) - 1000)

woa = woa.rename({'thkcello':'dz_t'})
woa = woa.chunk({'lev':-1,'x':180})

#compute omz_thickness
woa['omz_thickness'] = omz_thickness(woa, o2_bins=o2_bins)

In [15]:
#transform to sigma-space
woa_sigma = transform_wrapper(woa, 
                             intensive_vars=[va for va in woa.data_vars if va not in ['omz_thickness']],
                             sigma_bins = fine_sigma_bins)

In [16]:
# confirm that the omz volume is conserved
with ProgressBar():
    vol_sigma = (woa_sigma.omz_thickness * woa_sigma.areacello).sum(['x','y','sigma_0']).load()
    vol_z = (woa.omz_thickness * woa.areacello).sum(['x', 'y', 'lev']).load()
assert (((vol_z - vol_sigma)/vol_z*100)<1.0).all() #failed 0.5%, change to 1%

[########################################] | 100% Completed |  1min 17.2s
[########################################] | 100% Completed |  1.6s


In [8]:
fname = ofolder.joinpath('woa13_fine_sigma.nc')
woa_sigma = write(woa_sigma, fname, overwrite=True)

Removing file ../../data/external/fine_density_WOA/woa13_fine_sigma.nc
$ Saving 5.23792338GB to ../../data/external/fine_density_WOA/woa13_fine_sigma.nc


/home/samjd/anaconda3/envs/cmip6_omz2/lib/python3.9/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/samjd/anaconda3/envs/cmip6_omz2/lib/python3.9/site-packages/numba/np/ufunc/gufunc.py:151: RuntimeWarning: invalid value encountered in _interp_1d_linear
  return self.ufunc(*args, **kwargs)


$ Reloading file


In [9]:
woa_sigma

<xarray.Dataset>
Dimensions:             (bnds: 2, o2_bin: 6, sigma_0: 45, x: 1440, y: 720)
Coordinates: (12/13)
    crs                 int32 ...
  * y                   (y) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * x                   (x) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
    lon                 (x, y) float32 ...
    lat                 (x, y) float32 ...
    areacello           (x, y) float32 ...
    ...                  ...
  * o2_bin              (o2_bin) int64 10 40 60 80 100 120
    climatology_bounds  (bnds) float32 ...
    lev                 (y, x, sigma_0) float64 ...
    dz_t                (y, x, sigma_0) float64 ...
    lev_bounds          (bnds, y, x, sigma_0) float64 ...
    sigma_0_bounds      (bnds, sigma_0) float64 ...
Dimensions without coordinates: bnds
Data variables:
    so                  (y, x, sigma_0) float64 ...
    thetao              (y, x, sigma_0) float64 ...
    o2                  (y, x, sigma_0) float64 ...
    aou                 (y, x, sigma_0) float64 ...
    po4                 (y, x, sigma_0) float64 ...
    no3                 (y, x, sigma_0) float64 ...
    o2sat               (y, x, sigma_0) float64 ...
    omz_thickness       (o2_bin, y, x, sigma_0) float32 ...
Attributes: (12/45)
    Conventions:                     CF-1.6
    title:                           World Ocean Atlas 2013 version 2 : sea_w...
    summary:                         Climatological mean salinity for the glo...
    references:                      Zweng, M. M, J. R. Reagan, J. I. Antonov...
    institution:                     National Oceanographic Data Center(NODC)
    comment:                         global climatology as part of the World ...
    ...                              ...
    nodc_template_version:           NODC_NetCDF_Grid_Template_v1.0
    license:                         These data are openly available to the p...
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    metadata_link:                   http://www.nodc.noaa.gov/OC5/WOA13/pr_wo...
    date_created:                    2011-09-28 
    date_modified:                   2011-09-28

In [ ]:
# #If you want to save the processed version in z-space...
# fname = ofolder.joinpath('woa13_z_space.nc')
# woa_z = write(woa, fname, overwrite=True)